In [1]:
import tushare as ts
import pandas as pd
pd.set_option('display.max_columns', None)

import shutil
import os

start_year = 2009
gap_year = 9  # 9年作为训练，之后一年用来测试， 2009-2018 2019
pro = pro = ts.pro_api(
    '19c1ab37c30ca784d8658cf1050c0aba8eef6cb8d7cda5d0744d6bfb')

In [ ]:
columns_to_drop = ['ts_code', 'change', 'pct_chg', 'amount']  # 删除指定的列

if not os.path.exists('dataset'):
    os.mkdir('dataset')
else:
    shutil.rmtree('dataset')
    os.mkdir('dataset')

df = pro.daily(ts_code='000001.SZ', start_date='20090101', end_date='20240101')
df.drop(columns=columns_to_drop, inplace=True)
df['trade_date'] = pd.to_datetime(df["trade_date"], format="%Y%m%d")
df_reversed = df[::-1]
df_reversed.to_csv('./dataset/reversed_all_data.csv',
                   index=False, header=False)  # index=False 防止保存索引列

In [2]:
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
data.to_csv('./data')

In [3]:
df = pro.daily(ts_code='000700.SZ', start_date='20050401', end_date='20240401')


In [5]:
import pathlib
file_path = pathlib.Path('./dataset/graph_label/reversed_all_data.csv')

In [8]:
file_path.resolve

<bound method Path.resolve of WindowsPath('dataset/graph_label/reversed_all_data.csv')>

In [9]:
os.mkdir(file_path.parent)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'dataset\\graph_label'

In [11]:
file_path/'shs'

WindowsPath('dataset/graph_label/reversed_all_data.csv/shs')

In [12]:
from utils.formula import *

In [13]:
import os 
os.getcwd()

'c:\\Users\\HX\\Documents\\Graduation Project\\code\\graduation_project'

In [19]:
import numpy as np

def calculate_return_labels(prices, theta=0.2, t_forward=1):
    
    if len(prices) < t_forward + 1:
        raise ValueError("ERROE: PRICE LENGTH TOO SHORT")
    
    future_prices = prices[t_forward:]
    log_returns = np.log(future_prices) - np.log(prices[:-t_forward])
    
    sorted_returns = np.sort(log_returns)[::-1]
    idx_theta = int(len(sorted_returns) * theta)
    idx_one_minus_theta = int(len(sorted_returns) * (1 - theta))
    r_theta = sorted_returns[idx_theta]
    r_one_minus_theta = sorted_returns[idx_one_minus_theta]
    
    labels = []
    for ret in log_returns:
        if ret > r_theta:
            labels.append(1)
        elif ret < r_one_minus_theta:
            labels.append(-1)
        else:
            labels.append(0)
    
    return labels

prices_example = [6.1, 6.2, 6.4, 6.3, 6.5, 6.7, 6.8]

# 计算标签
labels_result = calculate_return_labels(prices_example, theta=0.2, t_forward=1)
print(labels_result)

[0, 1, -1, 0, 0, 0]


In [19]:
# This is a new version of phase1 constructed with pandas_ta
import os
import pathlib
import pandas as pd
import pandas_ta as ta
from utils.formula import *
from sklearn.preprocessing import MinMaxScaler
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

start = 6
end = 21

column_names = ["date", "open", "high", "low",
                "close", "preClose", "vol"]  # Define column names

file_path = pathlib.Path('./dataset/reversed_all_data.csv')
dir_path = file_path.parent
source_data = pd.read_csv(file_path, header=None, names=column_names)
source_data['date'] = pd.to_datetime(
    source_data["date"], format="%Y-%m-%d")
source_data.set_index('date', inplace=True)


In [22]:
LABELS_list = calculate_trichotomous_LABELS(source_data['close'].tolist(), t_forward=4)

source_data.drop(source_data.head(4).index, inplace=True)

source_data['LABELS'] = LABELS_list
len(source_data)

2790

In [23]:

source_data

,open,high,low,close,preClose,vol,LABELS
date,,,,,,,
2012-04-12,6.06,6.19,6.02,6.19,6.07,26472.53,1
2012-04-13,6.19,6.27,6.15,6.19,6.19,22851.07,1
2012-04-16,6.12,6.35,6.12,6.29,6.19,28277.61,1
2012-04-17,6.25,6.34,6.11,6.14,6.29,22915.47,0
2012-04-18,6.11,6.34,6.11,6.30,6.14,28686.56,0
...,...,...,...,...,...,...,...
2024-03-26,7.00,7.70,6.92,7.64,7.01,1158186.56,1
2024-03-27,7.36,7.38,6.88,6.88,7.64,717638.47,0
2024-03-28,6.66,7.23,6.66,7.09,6.88,567817.67,1
